# Comparing Supervised and Self-Supervised Approaches

Обучение всех моделей происходило с помощью фреймворка Pytorch Lightning. Супер вещь!

Файлы с кодом доступны в [репо](https://github.com/voorhs/byol)

Веса доступны на [диске](https://drive.google.com/drive/folders/1QBKd0jUGGpLmT20RqoFmqNq4hJpMvHtY?usp=drive_link)

## Project Embeddings onto Plane

### Supervised

Используется модель на основе ResNet, описана в файле `net.py`.

In [2]:
from train_supervised import SupervisedLearner
from net import MyResNet
import torch


# network for supervised learning
net = MyResNet()

# load checkpoint
supervised_model = SupervisedLearner.load_from_checkpoint(
    "lightning_logs/supervised/checkpoints/last.ckpt",
    net=net
)

In [3]:
# the layer before fc classifier
layer = dict(supervised_model.named_modules())['learner.max_pool']


# hook that saves output of avgpool
def _hook( _, __, output):
    global sup_embeddings_list
    sup_embeddings_list.append(torch.flatten(output, start_dim=1).cpu().detach().numpy())

# register hook
handle = layer.register_forward_hook(_hook)

In [4]:
from torchvision.datasets import CIFAR100
import torchvision.transforms as T


# load data
def load_cifar100(train):
    res = CIFAR100(
        root='./data',
        train=train,
        download=True,
        transform=T.ToTensor()
    )
    return res

In [5]:
from torch.utils.data import DataLoader

test_cifar100 = load_cifar100(train=False)

test_cifar100_loader = DataLoader(
    dataset=test_cifar100,
    batch_size=1024,
    shuffle=False,
    num_workers=10
)

Files already downloaded and verified


In [6]:
import numpy as np

# index of class to its name
cifar100_itos = np.array([s.rstrip() for s in open('cifar100-labels.txt', 'r').readlines()])

# retrieve embeddings from trained BYOL
sup_embeddings_list = []
sup_images_list = []
sup_labels_list = []
sup_targets_list = []
for images, labels in test_cifar100_loader:
    # forward triggers hook that saves embeddings
    supervised_model(images.cuda())
    sup_images_list.append(images.numpy())
    sup_labels_list.append(cifar100_itos[labels])
    sup_targets_list.append(labels)

# to numpy
sup_embeddings = np.concatenate(sup_embeddings_list)
sup_images = np.concatenate(sup_images_list)
sup_labels = np.concatenate(sup_labels_list)
sup_targets = np.concatenate(sup_targets_list)

sup_embeddings.shape, sup_images.shape, sup_labels.shape, sup_targets.shape

((10000, 1028), (10000, 3, 32, 32), (10000,), (10000,))

Интерактивная визуализация: при наведении мышки на точку всплывает изображение и имя класса.

In [6]:
from on_plane import visualize

app = visualize(sup_embeddings, sup_images, sup_labels, sup_targets)

if __name__ == "__main__":
    # this may take a while if dataset is large
    app.run_server(debug=True, mode='inline', port=8050)

Dash is running on http://127.0.0.1:8050/



Выводы:
- видим, что отдельные классы формируют кластеры
- особенно это заметно на краях проекции, в них часто попадают простые и однотипные примеры, которые хорошо отделяются моделью от остальных объектов выборки (все -- на белом фоне, все -- деревья, и т.п.)

### BYOL

Алгоритм из этой статьи: https://arxiv.org/abs/2006.07733. Он использует сеть той же архитектуры в качестве feature extractor, но учит не предсказывать метку класса, а приближать эмбеддинги двух аугментаций одного изображения.

In [7]:
from train_byol import SelfSupervisedLearner


# network for supervised learning
net = MyResNet()

# load checkpoint
selfsupervised_model = SelfSupervisedLearner.load_from_checkpoint(
    'lightning_logs/byol_batch256_lr5e-4/checkpoints/last.ckpt',
    net=net,
    image_size=32,
    hidden_layer='max_pool',
    projection_size=128,
    projection_hidden_size=512,
    moving_average_decay=0.99
)

/home/ilya/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning:

The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).



In [8]:
import numpy as np

# index of class to its name
cifar100_itos = np.array([s.rstrip() for s in open('cifar100-labels.txt', 'r').readlines()])

# retrieve embeddings from trained BYOL
byol_embeddings_list = []
byol_images_list = []
byol_labels_list = []
byol_targets_list = []
for images, labels in test_cifar100_loader:
    # forward triggers hook that saves embeddings
    byol_embeddings_list.append(selfsupervised_model.learner.online_encoder.get_representation(images.cuda()).cpu().detach().numpy())
    byol_images_list.append(images.numpy())
    byol_labels_list.append(cifar100_itos[labels])
    byol_targets_list.append(labels)

# to numpy
byol_embeddings = np.concatenate(byol_embeddings_list)
byol_images = np.concatenate(byol_images_list)
byol_labels = np.concatenate(byol_labels_list)
byol_targets = np.concatenate(byol_targets_list)

byol_embeddings.shape, byol_images.shape, byol_labels.shape, byol_targets.shape

((10000, 1028), (10000, 3, 32, 32), (10000,), (10000,))

Интерактивная визуализация

In [9]:
from on_plane import visualize

app = visualize(byol_embeddings, byol_images, byol_labels, byol_targets)

if __name__ == "__main__":
    # this may take a while if dataset is large
    app.run_server(debug=True, mode='inline', port=8051)

Dash is running on http://127.0.0.1:8051/



Выводы:
- видим, что по краям проекции снова выделились кластеры классов, они соответствуют простым и однотипным примерам, как и в случае с supervised методом выше
    - plain
    - apple на белом фоне
    - lawn mower на белом фоне
- при этом появились кластеры объектов из разных классов, но объединённые фоном:
    - акулы, киты, дельфины -- сняты под водой
    - поезда, танки, машины, верблюды -- на фоне пейзажа
    - всевозможные цветы
- в середине проекции видны кластеры из
    - животных
    - техники (автомобили, поезда ...)
    - человеческих лиц

### Random Weights

Случайно инициализированный ResNet (все той же архитектуры)

In [10]:
# network for supervised learning
net = MyResNet()

# load checkpoint
supervised_model = SupervisedLearner(net).cuda()

In [11]:
# the layer before fc classifier
layer = dict(supervised_model.named_modules())['learner.max_pool']

# hook that saves output of avgpool
def _hook( _, __, output):
    global rand_embeddings_list
    rand_embeddings_list.append(torch.flatten(output, start_dim=1).cpu().detach().numpy())

# register hook
handle = layer.register_forward_hook(_hook)

In [12]:
import numpy as np

# index of class to its name
cifar100_itos = np.array([s.rstrip() for s in open('cifar100-labels.txt', 'r').readlines()])

# retrieve embeddings from trained BYOL
rand_embeddings_list = []
rand_images_list = []
rand_labels_list = []
rand_targets_list = []
for images, labels in test_cifar100_loader:
    # forward triggers hook that saves embeddings
    supervised_model(images.cuda())
    rand_images_list.append(images.numpy())
    rand_labels_list.append(cifar100_itos[labels])
    rand_targets_list.append(labels)

# to numpy
rand_embeddings = np.concatenate(rand_embeddings_list)
rand_images = np.concatenate(rand_images_list)
rand_labels = np.concatenate(rand_labels_list)
rand_targets = np.concatenate(rand_targets_list)

rand_embeddings.shape, rand_images.shape, rand_labels.shape, rand_targets.shape

((10000, 1028), (10000, 3, 32, 32), (10000,), (10000,))

In [13]:
from on_plane import visualize

app = visualize(rand_embeddings, rand_images, rand_labels, rand_targets)

if __name__ == "__main__":
    # this may take a while if dataset is large
    app.run_server(debug=True, mode='inline', port=8052)

Dash is running on http://127.0.0.1:8052/



Выводы:
- По краям все так же видны кластеры с однотипными примерами, но эти кластеры меньше и их количество меньше
- В середине проекции разглядеть структуру сложнее
- Значит даже случайные свёртки не бесполезны для извлечения признаков

## Linear Evaluation Protocol

Оценим эффективность эмбеддингов, обучив на них OneVsRestClassifier над логистической регрессией.

In [11]:
from warnings import filterwarnings
filterwarnings('ignore')

### Supervised

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(sup_embeddings, sup_targets, random_state=0)
sup_clf = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)
print('accuracy:', sup_clf.score(X_test, y_test))

accuracy: 0.5908


### BYOL

In [16]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(byol_embeddings, byol_targets, random_state=0)
byol_clf = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)
print('accuracy:', byol_clf.score(X_test, y_test))

accuracy: 0.3164


### Random Weights

In [17]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(rand_embeddings, rand_targets, random_state=0)
rand_clf = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)
print('accuracy:', rand_clf.score(X_test, y_test))

accuracy: 0.1612


Выводы:
- видим, что самообучение не бесполезно!

## Улучшения

Можно убрать EMA из обновления весов таргет сети, как предложено в статье https://arxiv.org/abs/2011.10566

In [7]:
from train_byol import SelfSupervisedLearner


# network for supervised learning
net = MyResNet()

# load checkpoint
sims_supervised_model = SelfSupervisedLearner.load_from_checkpoint(
    'lightning_logs/byol_nomomentum/checkpoints/epoch=29-step=5880.ckpt',
    net=net,
    image_size=32,
    hidden_layer='max_pool',
    projection_size=128,
    projection_hidden_size=512,
    moving_average_decay=0.99,
    use_momentum=False
)

/home/ilya/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [8]:
# index of class to its name
cifar100_itos = np.array([s.rstrip() for s in open('cifar100-labels.txt', 'r').readlines()])

# retrieve embeddings from trained BYOL
sims_embeddings_list = []
sims_images_list = []
sims_labels_list = []
sims_targets_list = []
for images, labels in test_cifar100_loader:
    # forward triggers hook that saves embeddings
    sims_embeddings_list.append(sims_supervised_model.learner.online_encoder.get_representation(images.cuda()).cpu().detach().numpy())
    sims_images_list.append(images.numpy())
    sims_labels_list.append(cifar100_itos[labels])
    sims_targets_list.append(labels)

# to numpy
sims_embeddings = np.concatenate(sims_embeddings_list)
sims_images = np.concatenate(sims_images_list)
sims_labels = np.concatenate(sims_labels_list)
sims_targets = np.concatenate(sims_targets_list)

sims_embeddings.shape, sims_images.shape, sims_labels.shape, sims_targets.shape

((10000, 1028), (10000, 3, 32, 32), (10000,), (10000,))

In [9]:
from on_plane import visualize

app = visualize(sims_embeddings, sims_images, sims_labels, sims_targets)

if __name__ == "__main__":
    # this may take a while if dataset is large
    app.run_server(debug=True, mode='inline', port=8053)

Dash is running on http://127.0.0.1:8053/



In [12]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(sims_embeddings, sims_targets, random_state=0)
sims_clf = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)
print('accuracy:', sims_clf.score(X_test, y_test))

accuracy: 0.2432


## Что нужно было знать

- **Свёрточные сети**. Первые слои ResNet в оригинальных архитектурах стремительно сокращают размерность изображения, поскольку в imagenet 224*224. Для CIFAR-100 такие модели давали плохое качество (было сложнее обучить >50% accuracy), поэтому пришлось самому подбирать архитектуру, чтобы было разумно сравнивать подходы
- **Обучение с подкрепленем**. Идея сиамских близнецов взята из RL, поэтому знание простейших алгоритмов Deep RL помогло понять, почему BYOL работает
- **Стохастическая оптимизация**. Я долго экспериментировал с оптимизаторами и их параметрами, чтобы добиться приемлемого результата как для supervised, так и для self supervised
- **Mixed Precision** для ускорения обучения. Без этого обучалось значительно дольше
- **Manifold learning**. Для понимания, как интерпретировать визуализацию пространства эмбеддингов
- **Flask, порты**. Чтобы завести визуализацию, в которой всплывают изображение и имя класса для каждой точки на scatter plot